<a href="https://colab.research.google.com/github/kiaerii/lab.subbota/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
"""
Лабораторная работа: "Работа с базами данных в Python"
Реализация всех заданий
"""

import sqlite3
from datetime import datetime
from typing import Optional, List, Dict, Any


#ЗАДАНИЕ 1: Создание базы данных и таблиц

def create_database():
    """
    Создает базу данных "university.db" с таблицами students и courses
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            # Создание таблицы students
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS students (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    group_name TEXT NOT NULL,
                    admission_year INTEGER NOT NULL,
                    average_grade REAL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    CHECK (admission_year >= 2000 AND admission_year <= 2030),
                    CHECK (average_grade >= 0 AND average_grade <= 5)
                )
            ''')

            # Создание таблицы courses
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS courses (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    course_name TEXT UNIQUE NOT NULL,
                    instructor TEXT NOT NULL,
                    credits INTEGER NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    CHECK (credits > 0 AND credits <= 10)
                )
            ''')

            print("База данных и таблицы успешно созданы!")

    except sqlite3.Error as e:
        print(f"Ошибка при создании базы данных: {e}")


#ЗАДАНИЕ 2: CRUD-операции для таблицы студентов

def add_student(first_name: str, last_name: str, group_name: str,
                admission_year: int, average_grade: Optional[float] = None) -> Optional[int]:
    """
    Добавляет нового студента в базу данных
    """
    try:
        with sqlite3.connect('university.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('''
                INSERT INTO students (first_name, last_name, group_name, admission_year, average_grade)
                VALUES (?, ?, ?, ?, ?)
            ''', (first_name, last_name, group_name, admission_year, average_grade))

            conn.commit()
            return cursor.lastrowid

    except sqlite3.Error as e:
        print(f"Ошибка при добавлении студента: {e}")
        return None


def get_all_students() -> List[Dict[str, Any]]:
    """
    Возвращает список всех студентов
    """
    try:
        with sqlite3.connect('university.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('SELECT * FROM students ORDER BY last_name, first_name')
            students = cursor.fetchall()

            return [dict(student) for student in students]

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов: {e}")
        return []


def get_students_by_group(group_name: str) -> List[Dict[str, Any]]:
    """
    Возвращает студентов указанной группы
    """
    try:
        with sqlite3.connect('university.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('''
                SELECT * FROM students
                WHERE group_name = ?
                ORDER BY last_name, first_name
            ''', (group_name,))

            students = cursor.fetchall()
            return [dict(student) for student in students]

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов группы: {e}")
        return []


def update_student_grade(student_id: int, new_grade: float) -> bool:
    """
    Обновляет средний балл студента
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            cursor.execute('''
                UPDATE students
                SET average_grade = ?
                WHERE id = ?
            ''', (new_grade, student_id))

            conn.commit()
            return cursor.rowcount > 0

    except sqlite3.Error as e:
        print(f"Ошибка при обновлении оценки: {e}")
        return False


def delete_student(student_id: int) -> bool:
    """
    Удаляет студента по ID
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            cursor.execute('DELETE FROM students WHERE id = ?', (student_id,))

            conn.commit()
            return cursor.rowcount > 0

    except sqlite3.Error as e:
        print(f"Ошибка при удалении студента: {e}")
        return False


#ЗАДАНИЕ 3: Работа с транзакциями и связями

def create_relations_tables():
    """
    Создает таблицу для связи студентов и курсов
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            # Создание таблицы student_courses
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS student_courses (
                    student_id INTEGER NOT NULL,
                    course_id INTEGER NOT NULL,
                    enrollment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    PRIMARY KEY (student_id, course_id),
                    FOREIGN KEY (student_id) REFERENCES students(id) ON DELETE CASCADE,
                    FOREIGN KEY (course_id) REFERENCES courses(id) ON DELETE CASCADE
                )
            ''')

            # Включаем поддержку внешних ключей
            cursor.execute('PRAGMA foreign_keys = ON')

            print("Таблица связей успешно создана!")

    except sqlite3.Error as e:
        print(f"Ошибка при создании таблицы связей: {e}")


def add_course(course_name: str, instructor: str, credits: int) -> Optional[int]:
    """
    Добавляет новый курс
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            cursor.execute('''
                INSERT INTO courses (course_name, instructor, credits)
                VALUES (?, ?, ?)
            ''', (course_name, instructor, credits))

            conn.commit()
            return cursor.lastrowid

    except sqlite3.IntegrityError:
        print(f"Курс '{course_name}' уже существует!")
        return None
    except sqlite3.Error as e:
        print(f"Ошибка при добавлении курса: {e}")
        return None


def enroll_student_in_course(student_id: int, course_id: int) -> bool:
    """
    Записывает студента на курс
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            cursor.execute('''
                INSERT INTO student_courses (student_id, course_id)
                VALUES (?, ?)
            ''', (student_id, course_id))

            conn.commit()
            return cursor.rowcount > 0

    except sqlite3.IntegrityError:
        print("Студент уже записан на этот курс!")
        return False
    except sqlite3.Error as e:
        print(f"Ошибка при записи на курс: {e}")
        return False


def get_student_courses(student_id: int) -> List[Dict[str, Any]]:
    """
    Возвращает курсы студента
    """
    try:
        with sqlite3.connect('university.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('''
                SELECT c.*, sc.enrollment_date
                FROM courses c
                JOIN student_courses sc ON c.id = sc.course_id
                WHERE sc.student_id = ?
                ORDER BY c.course_name
            ''', (student_id,))

            courses = cursor.fetchall()
            return [dict(course) for course in courses]

    except sqlite3.Error as e:
        print(f"Ошибка при получении курсов студента: {e}")
        return []


def get_course_students(course_id: int) -> List[Dict[str, Any]]:
    """
    Возвращает студентов курса
    """
    try:
        with sqlite3.connect('university.db') as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('''
                SELECT s.*, sc.enrollment_date
                FROM students s
                JOIN student_courses sc ON s.id = sc.student_id
                WHERE sc.course_id = ?
                ORDER BY s.last_name, s.first_name
            ''', (course_id,))

            students = cursor.fetchall()
            return [dict(student) for student in students]

    except sqlite3.Error as e:
        print(f"Ошибка при получении студентов курса: {e}")
        return []


def transfer_student(student_id: int, new_group: str) -> bool:
    """
    Переводит студента в другую группу с использованием транзакции
    """
    try:
        with sqlite3.connect('university.db') as conn:
            cursor = conn.cursor()

            # Начинаем транзакцию
            conn.execute("BEGIN TRANSACTION")

            try:
                # Проверяем существование студента
                cursor.execute('SELECT id FROM students WHERE id = ?', (student_id,))
                if not cursor.fetchone():
                    print(f"Студент с ID {student_id} не найден!")
                    conn.rollback()
                    return False

                # Обновляем группу студента
                cursor.execute('''
                    UPDATE students
                    SET group_name = ?
                    WHERE id = ?
                ''', (new_group, student_id))

                # В реальном приложении здесь была бы логика
                # перезаписи на курсы новой группы

                conn.commit()
                print(f"Студент успешно переведен в группу {new_group}")
                return True

            except sqlite3.Error as e:
                conn.rollback()
                raise e

    except sqlite3.Error as e:
        print(f"Ошибка при переводе студента: {e}")
        return False


#ЗАДАНИЕ 4: Класс-обертка для работы с БД

class UniversityDB:
    """
    Класс-обертка для работы с базой данных университета
    """

    def __init__(self, db_path: str = 'university.db'):
        self.db_path = db_path
        self.connection = None
        self.cursor = None

    def __enter__(self):
        """Открывает соединение с базой данных"""
        self.connection = sqlite3.connect(self.db_path)
        self.connection.row_factory = sqlite3.Row
        self.cursor = self.connection.cursor()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Закрывает соединение с базой данных"""
        if self.connection:
            if exc_type is None:
                self.connection.commit()
            else:
                print(f"Произошла ошибка: {exc_val}")
                self.connection.rollback()
            self.connection.close()

    def execute_query(self, query: str, params: Optional[tuple] = None) -> bool:
        """Выполняет запрос без возврата результата"""
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            return True
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return False

    def fetch_all(self, query: str, params: Optional[tuple] = None) -> List[Dict[str, Any]]:
        """Выполняет запрос и возвращает все результаты"""
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)

            results = self.cursor.fetchall()
            return [dict(row) for row in results]
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return []

    def fetch_one(self, query: str, params: Optional[tuple] = None) -> Optional[Dict[str, Any]]:
        """Выполняет запрос и возвращает одну запись"""
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)

            result = self.cursor.fetchone()
            return dict(result) if result else None
        except sqlite3.Error as e:
            print(f"Ошибка выполнения запроса: {e}")
            return None

    def get_student_statistics(self) -> Dict[str, Any]:
        """Возвращает статистику по студентам"""
        try:
            statistics = {}

            # Общее количество студентов
            self.cursor.execute('SELECT COUNT(*) as total FROM students')
            statistics['total_students'] = self.cursor.fetchone()['total']

            # Средний балл по университету
            self.cursor.execute('SELECT AVG(average_grade) as avg_grade FROM students WHERE average_grade IS NOT NULL')
            result = self.cursor.fetchone()
            statistics['university_average_grade'] = round(result['avg_grade'], 2) if result['avg_grade'] else 0

            # Количество студентов по группам
            self.cursor.execute('''
                SELECT group_name, COUNT(*) as count
                FROM students
                GROUP BY group_name
                ORDER BY count DESC
            ''')
            statistics['students_by_group'] = self.fetch_all(
                'SELECT group_name, COUNT(*) as count FROM students GROUP BY group_name ORDER BY count DESC'
            )

            return statistics

        except sqlite3.Error as e:
            print(f"Ошибка при получении статистики: {e}")
            return {}

    def get_top_students(self, limit: int = 5) -> List[Dict[str, Any]]:
        """Возвращает топ студентов по среднему баллу"""
        return self.fetch_all('''
            SELECT id, first_name, last_name, group_name, average_grade
            FROM students
            WHERE average_grade IS NOT NULL
            ORDER BY average_grade DESC
            LIMIT ?
        ''', (limit,))

    def get_group_statistics(self) -> Dict[str, Any]:
        """Возвращает статистику по группам"""
        statistics = {}

        # Средний балл по группам
        statistics['average_grades_by_group'] = self.fetch_all('''
            SELECT group_name, ROUND(AVG(average_grade), 2) as avg_grade
            FROM students
            WHERE average_grade IS NOT NULL
            GROUP BY group_name
            ORDER BY avg_grade DESC
        ''')

        # Количество студентов в каждой группе
        statistics['students_per_group'] = self.fetch_all('''
            SELECT group_name, COUNT(*) as student_count
            FROM students
            GROUP BY group_name
            ORDER BY student_count DESC
        ''')

        return statistics

    def search_students_by_last_name(self, last_name_part: str) -> List[Dict[str, Any]]:
        """Ищет студентов по части фамилии"""
        return self.fetch_all('''
            SELECT * FROM students
            WHERE last_name LIKE ?
            ORDER BY last_name, first_name
        ''', (f'%{last_name_part}%',))


#ЗАДАНИЕ 5: Комплексное приложение с интерфейсом

def print_student_table(students: List[Dict[str, Any]]):
    """Красиво выводит таблицу студентов"""
    if not students:
        print("Нет данных для отображения")
        return

    # Определяем ширину колонок
    headers = ["ID", "Имя", "Фамилия", "Группа", "Год", "Средний балл"]
    col_widths = [4, 15, 15, 10, 6, 12]

    # Выводим заголовок
    header_row = " | ".join(h.ljust(w) for h, w in zip(headers, col_widths))
    print("\n" + "=" * len(header_row))
    print(header_row)
    print("=" * len(header_row))

    # Выводим данные
    for student in students:
        row = [
            str(student.get('id', '')),
            student.get('first_name', ''),
            student.get('last_name', ''),
            student.get('group_name', ''),
            str(student.get('admission_year', '')),
            f"{student.get('average_grade', '') or 'Н/Д'}"
        ]
        print(" | ".join(r.ljust(w) for r, w in zip(row, col_widths)))

    print("=" * len(header_row))


def add_student_interactive():
    """Интерактивное добавление студента"""
    print("\n--- Добавление студента ---")

    first_name = input("Имя: ").strip()
    if not first_name:
        print("Имя не может быть пустым!")
        return

    last_name = input("Фамилия: ").strip()
    if not last_name:
        print("Фамилия не может быть пустой!")
        return

    group_name = input("Группа (например, ГР-01): ").strip()
    if not group_name:
        print("Группа не может быть пустой!")
        return

    # Валидация года поступления
    while True:
        try:
            admission_year = int(input("Год поступления (2000-2030): ").strip())
            if 2000 <= admission_year <= 2030:
                break
            else:
                print("Год должен быть между 2000 и 2030")
        except ValueError:
            print("Введите корректный год")

    # Валидация среднего балла
    average_grade = None
    grade_input = input("Средний балл (0-5, оставьте пустым если нет): ").strip()
    if grade_input:
        try:
            grade = float(grade_input)
            if 0 <= grade <= 5:
                average_grade = grade
            else:
                print("Балл должен быть между 0 и 5. Будет установлено значение NULL")
        except ValueError:
            print("Некорректный формат балла. Будет установлено значение NULL")

    # Добавление в базу данных
    student_id = add_student(first_name, last_name, group_name, admission_year, average_grade)
    if student_id:
        print(f"Студент успешно добавлен с ID {student_id}!")
    else:
        print("Ошибка при добавлении студента")


def display_all_students():
    """Отображает всех студентов"""
    print("\n--- Все студенты ---")

    with UniversityDB('university.db') as db:
        students = db.fetch_all('SELECT * FROM students ORDER BY last_name, first_name')
        print_student_table(students)
        print(f"\nВсего студентов: {len(students)}")


def search_student_interactive():
    """Поиск студента по фамилии"""
    print("\n--- Поиск студента по фамилии ---")

    last_name_part = input("Введите часть фамилии для поиска: ").strip()
    if not last_name_part:
        print("Введите хотя бы один символ для поиска!")
        return

    with UniversityDB('university.db') as db:
        students = db.search_students_by_last_name(last_name_part)

        if students:
            print(f"\nНайдено студентов: {len(students)}")
            print_student_table(students)
        else:
            print("Студенты не найдены")


def update_grade_interactive():
    """Обновление оценки студента"""
    print("\n--- Обновление оценки студента ---")

    try:
        student_id = int(input("Введите ID студента: ").strip())
    except ValueError:
        print("Некорректный ID!")
        return

    # Проверяем существование студента
    with UniversityDB('university.db') as db:
        student = db.fetch_one('SELECT * FROM students WHERE id = ?', (student_id,))

        if not student:
            print(f"Студент с ID {student_id} не найден!")
            return

        print(f"\nСтудент: {student['first_name']} {student['last_name']}")
        print(f"Текущий средний балл: {student['average_grade'] or 'Н/Д'}")

        # Валидация нового балла
        while True:
            try:
                new_grade = input("Новый средний балл (0-5): ").strip()
                if not new_grade:
                    print("Операция отменена")
                    return

                new_grade_float = float(new_grade)
                if 0 <= new_grade_float <= 5:
                    break
                else:
                    print("Балл должен быть между 0 и 5")
            except ValueError:
                print("Введите корректное число")

        # Обновляем оценку
        success = update_student_grade(student_id, new_grade_float)
        if success:
            print("Оценка успешно обновлена!")
        else:
            print("Ошибка при обновлении оценки")


def delete_student_interactive():
    """Удаление студента"""
    print("\n--- Удаление студента ---")

    try:
        student_id = int(input("Введите ID студента для удаления: ").strip())
    except ValueError:
        print("Некорректный ID!")
        return

    # Показываем информацию о студенте
    with UniversityDB('university.db') as db:
        student = db.fetch_one('SELECT * FROM students WHERE id = ?', (student_id,))

        if not student:
            print(f"Студент с ID {student_id} не найден!")
            return

        print(f"\nВы собираетесь удалить студента:")
        print(f"ID: {student['id']}")
        print(f"ФИО: {student['first_name']} {student['last_name']}")
        print(f"Группа: {student['group_name']}")

        confirm = input("\nВы уверены? (да/НЕТ): ").strip().lower()
        if confirm == 'да':
            success = delete_student(student_id)
            if success:
                print("Студент успешно удален!")
            else:
                print("Ошибка при удалении студента")
        else:
            print("Удаление отменено")


def show_statistics():
    """Показывает статистику"""
    print("\n--- Статистика университета ---")

    with UniversityDB('university.db') as db:
        stats = db.get_student_statistics()

        print(f"\nОбщая статистика:")
        print(f"Всего студентов: {stats.get('total_students', 0)}")
        print(f"Средний балл по университету: {stats.get('university_average_grade', 0)}")

        print(f"\nРаспределение по группам:")
        for group in stats.get('students_by_group', []):
            print(f"  {group['group_name']}: {group['count']} студентов")

        # Топ студентов
        print(f"\nТоп-5 студентов по успеваемости:")
        top_students = db.get_top_students(5)
        for i, student in enumerate(top_students, 1):
            print(f"{i}. {student['first_name']} {student['last_name']} "
                  f"({student['group_name']}): {student['average_grade']}")


def main():
    """Главная функция приложения"""
    # Инициализация базы данных
    create_database()
    create_relations_tables()


    print("      СИСТЕМА УЧЕТА СТУДЕНТОВ УНИВЕРСИТЕТА")


    while True:
        print("\nГЛАВНОЕ МЕНЮ")
        print("1. Добавить студента")
        print("2. Просмотреть всех студентов")
        print("3. Найти студента по фамилии")
        print("4. Обновить оценку студента")
        print("5. Удалить студента")
        print("6. Показать статистику")
        print("7. Тестовые данные")
        print("8. Выход")

        choice = input("\nВыберите действие (1-8): ").strip()

        if choice == '1':
            add_student_interactive()
        elif choice == '2':
            display_all_students()
        elif choice == '3':
            search_student_interactive()
        elif choice == '4':
            update_grade_interactive()
        elif choice == '5':
            delete_student_interactive()
        elif choice == '6':
            show_statistics()
        elif choice == '7':
            create_test_data()
        elif choice == '8':
            print("\nДо свидания!")
            break
        else:
            print("Неверный выбор! Пожалуйста, выберите от 1 до 8.")


def create_test_data():
    """Создает тестовые данные для демонстрации"""
    print("\n--- Создание тестовых данных ---")

    # Добавляем студентов
    test_students = [
        ("Иван", "Петров", "ГР-01", 2023, 4.5),
        ("Мария", "Иванова", "ГР-01", 2023, 4.8),
        ("Алексей", "Сидоров", "ГР-02", 2023, 3.9),
        ("Елена", "Кузнецова", "ГР-02", 2022, 4.2),
        ("Дмитрий", "Васильев", "ГР-03", 2022, 4.7),
    ]

    student_ids = []
    for student in test_students:
        student_id = add_student(*student)
        if student_id:
            student_ids.append(student_id)

    # Добавляем курсы
    test_courses = [
        ("Математика", "Проф. Иванов", 5),
        ("Физика", "Проф. Петрова", 4),
        ("Программирование", "Доц. Сидоров", 6),
        ("История", "Доц. Кузнецова", 3),
    ]

    course_ids = []
    for course in test_courses:
        course_id = add_course(*course)
        if course_id:
            course_ids.append(course_id)

    # Записываем студентов на курсы
    enrollments = [
        (student_ids[0], course_ids[0]),  # Иван на Математику
        (student_ids[0], course_ids[2]),  # Иван на Программирование
        (student_ids[1], course_ids[0]),  # Мария на Математику
        (student_ids[1], course_ids[1]),  # Мария на Физику
        (student_ids[2], course_ids[1]),  # Алексей на Физику
        (student_ids[3], course_ids[2]),  # Елена на Программирование
        (student_ids[4], course_ids[3]),  # Дмитрий на История
    ]

    for enrollment in enrollments:
        enroll_student_in_course(*enrollment)

    print(f"Добавлено студентов: {len(student_ids)}")
    print(f"Добавлено курсов: {len(course_ids)}")
    print(f"Создано записей на курсы: {len(enrollments)}")
    print("\nТестовые данные успешно созданы!")

if __name__ == "__main__":
    main()

База данных и таблицы успешно созданы!
Таблица связей успешно создана!
      СИСТЕМА УЧЕТА СТУДЕНТОВ УНИВЕРСИТЕТА

ГЛАВНОЕ МЕНЮ
1. Добавить студента
2. Просмотреть всех студентов
3. Найти студента по фамилии
4. Обновить оценку студента
5. Удалить студента
6. Показать статистику
7. Тестовые данные
8. Выход

Выберите действие (1-8): 1

--- Добавление студента ---
Имя: Ольга
Фамилия: Булганина
Группа (например, ГР-01): ЗФИбд-01-24
Год поступления (2000-2030): 2024
Средний балл (0-5, оставьте пустым если нет): 5
Студент успешно добавлен с ID 1!

ГЛАВНОЕ МЕНЮ
1. Добавить студента
2. Просмотреть всех студентов
3. Найти студента по фамилии
4. Обновить оценку студента
5. Удалить студента
6. Показать статистику
7. Тестовые данные
8. Выход

Выберите действие (1-8): 2

--- Все студенты ---

ID   | Имя             | Фамилия         | Группа     | Год    | Средний балл
1    | Ольга           | Булганина       | ЗФИбд-01-24 | 2024   | 5.0         

Всего студентов: 1

ГЛАВНОЕ МЕНЮ
1. Добавить студен